In [2]:
import pandas as pd
import plotly as p
import plotly.express as px 
import plotly.io as pio
import plotly.graph_objects as go
import numpy as np
pio.templates.default = "plotly_dark"

In [201]:

years = list(map(str, list(range(2018, 2022, 1)))) # years with data available on BTS
quarters = ("Q1", "Q2", "Q3", "Q4")
denied_board_dfs = list()

# read in data that uses new reporting method - excel sheet with sheet for each quarter 
for year in years: 
    for quarter in quarters:
        file_path = "..\\raw_data\\denied_boarding\\Denied-Confirmed-Space-" + year + "-Operating-Carrier.xlsx"
        sheet = quarter + year
        df = pd.read_excel(file_path, sheet_name=sheet, header = 2)
        # column definitions changed in 2021
        if (year != "2021"):
            df[6] = df[7]
            df[7] = df['8(c)']
            df = df[['CARRIER',    
                     '1(a)',    
                     '1(b)',    
                     '2(a)',    
                     '2(b)',    
                     '2(c)',
                     3,         
                     4,         
                     5,
                     6,
                     7]]
        df['year'] = year
        df['quarter'] = quarter + " " + year
        df = df[pd.isna(df[6]) != True ]
        denied_board_dfs.append(df)
        
denied_boarding = pd.concat(denied_board_dfs)

denied_boarding[1] = denied_boarding['1(a)']+denied_boarding['1(b)']
denied_boarding[2] = denied_boarding['2(a)']+denied_boarding['2(b)']+denied_boarding['2(c)']
denied_boarding['perc_denied_boarding'] = round((denied_boarding[3] + denied_boarding[5])/denied_boarding[6], 6)*100
denied_boarding['perc_denied_boarding_involuntary'] = round(denied_boarding[3]/denied_boarding[6], 6)*100
denied_boarding['perc_denied_boarding_voluntary'] = round(denied_boarding[5]/denied_boarding[6], 6)*100

denied_boarding['date'] = pd.to_datetime([
    '-'.join(x.split()[::-1]) for x in denied_boarding['quarter']])

denied_boarding.to_csv("..\\clean_data\\denied_boarding.csv")

In [142]:
denied_boarding.CARRIER.unique()

array(['Alaska Airlines', 'Allegiant Airlines', 'American Airlines',
       'Delta Air Lines', 'Endeavor Air', 'Envoy Air',
       'ExpressJet Airlines', 'Frontier Airlines', 'Hawaiian Airlines',
       'JetBlue Airways', 'Mesa Airlines', 'PSA Airlines',
       'Republic Airline', 'SkyWest Airlines', 'Southwest Airlines',
       'Spirit Airlines', 'United Air Lines', 'Virgin America ',
       'Alaska Airlines Codeshare', 'Allegiant Air', 'Republic Airways',
       'United Airlines', 'Horizon Air'], dtype=object)

In [199]:
# create a figure for each category
figs = {
    c: px.area(denied_boarding[denied_boarding['CARRIER'] ==c]
               .rename(columns = {'perc_denied_boarding_involuntary':"Involuntary", 
                                                 "perc_denied_boarding_voluntary":"Voluntary"}), 
              x="date", y=["Voluntary","Involuntary"], 
                labels={
                    "date": "Date",
                     "value":"Percent of Passengers Denied Boarding", 
                     "variable":"Denial Type"
                 },
                title="Percent of Passengers Denied Boarding from 1990 to 2021", color_discrete_sequence=px.colors.qualitative.Dark24)
    
    for c in denied_boarding.CARRIER.unique()
}

# integrate figures per category into one figure
defaultcat = denied_boarding.CARRIER.unique()[0]
fig = figs[defaultcat].update_traces(visible=True)
for k in figs.keys():
    if k != defaultcat:
        fig.add_traces(figs[k].data)
        
# finally build dropdown menu
fig.update_layout(
    updatemenus=[
        dict(
            buttons = list([
                {
                    "label": k,
                    "method": "update",
                    # list comprehension for which traces are visible
                    "args": [{"visible": [kk == k for kk in figs.keys()]},
                             {"title":go.layout.xaxis.Title(
            text=f"Percent of Passengers Denied Boarding from 2018 to 2021 by Airline <br><sup>{k} </sup>"
           )}],
                }
                for k in figs.keys()
            ]), 
        x = 1
        ),
    ]
)

fig.show()


In [197]:
figs['Endeavor Air']

In [ ]:
denied_boarding = denied_boarding[denied_boarding['CARRIER']=='Alaska Airlines']
fig = px.area(denied_boarding.rename(columns = {'perc_denied_boarding_involuntary':"Involuntary", "perc_denied_boarding_voluntary":"Voluntary"}), 
              x="date", y=["Voluntary","Involuntary"], 
                labels={
                    "date": "Date",
                     "value":"Percent of Passengers Denied Boarding", 
                     "variable":"Denial Type"
                 },
                title="Percent of Passengers Denied Boarding from 1990 to 2021", color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_xaxes(tickangle = 45)
fig.show()